# Quick demonstration of pangeo tools applied to ICESat-2 data

## to create an environment to run this code:
conda create -n icesat python matplotlib h5py s3fs xarray geopandas hvplot geoviews netCDF4 seaborn pandas astropy spyder jupyter
- Use an ATLO6 Granule

In [4]:


import h5py
import s3fs
import xarray as xr
import os
import geopandas as gpd
import pandas as pd

#import geoviews as gv
#import hvplot.pandas
#import hvplot.xarray

In [6]:
print(xr.__version__)
print(s3fs.__version__)
print(gpd.__version__)

2022.11.0
2022.10.0
0.9.0


In [5]:
# Open a connection to the file on S3
s3 = s3fs.S3FileSystem()

In [10]:
fileobj = s3.open('./ATL06_20181014191048_02470103_005_01.h5')

ConnectTimeoutError: Connect timeout on endpoint URL: "http://169.254.169.254/latest/api/token"

In [7]:
# Read with h5py just as we've done for local files
fileobj = s3.open('pangeo-data-upload-oregon/icesat2/juneauicefield/ATL06_20190215180122_07530203_001_01.h5')
f = h5py.File(fileobj, 'r')
print(f)
print(f.keys())
print(f['gt1l/land_ice_segments/'].keys())
print(f['gt1l/land_ice_segments/h_li'])
print(f['gt1l/land_ice_segments/latitude'])
print(f['gt1l/land_ice_segments/longitude'])

ValueError: Attempt to open non key-like path: ATL06_20181014191048_02470103_005_01.h5

In [5]:
# Steam some data 
hli = f['gt1l/land_ice_segments/h_li'][:]
hli

array([650.07587  , 651.325    , 651.9      , ...,  -7.1765842,
        -7.1845217,  -7.1968102], dtype=float32)

In [6]:
# The data is tabular rather than gridded (~50000 multipoints with attributes per h5 file)
# NOTE this is not gridded data, it's suitable for tabular representation with geopandas
fileobj = s3.open('pangeo-data-upload-oregon/icesat2/juneauicefield/ATL06_20190215180122_07530203_001_01.h5')
df = pd.read_hdf(fileobj, 'gt1l/land_ice_segments') # not yet, but soon?

NotImplementedError: Support for generic buffers has not been implemented.

In [7]:
# Load with xarray
# Open a connection to the file on S3
fileobj = s3.open('pangeo-data-upload-oregon/icesat2/juneauicefield/ATL06_20190215180122_07530203_001_01.h5')
ds = xr.open_dataset(fileobj, engine='h5netcdf') # reads all the attributes by default but no data groups
ds

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes:
    granule_type:                       ATL06
    short_name:                         ATL06
    level:                              3A
    description:                        Land ice surface heights for each bea...
    contributor_name:                   Thomas E Neumann (thomas.neumann@nasa...
    contributor_role:                   Instrument Engineer, Investigator, Pr...
    Conventions:                        CF-1.6
    data_rate:                          Data within this group pertain to the...
    date_type:                          UTC
    featureType:                        trajectory
    geospatial_lat_units:               degrees_north
    geospatial_lon_units:               degrees_east
    identifier_product_doi:             10.5067/ATLAS/ATL06.001
    identifier_product_doi_authority:   http://dx.doi.org
    identifier_product_type:            ATL06
    license:                            Data may no

In [8]:
# Pull single group.

# Seems slower than pulling entire file to disk first, then loading?...
# Would be good to compare against Zarr or TileDB
fileobj = s3.open('pangeo-data-upload-oregon/icesat2/juneauicefield/ATL06_20190215180122_07530203_001_01.h5')
ds = xr.open_dataset(fileobj, group='gt1l/land_ice_segments', engine='h5netcdf', chunks=dict(delta_time=5000))
ds

<xarray.Dataset>
Dimensions:                (delta_time: 50045)
Coordinates:
  * delta_time             (delta_time) datetime64[ns] 2019-02-15T18:01:21.878254460 ... 2019-02-15T18:04:08.856824204
    latitude               (delta_time) float64 dask.array<shape=(50045,), chunksize=(5000,)>
    longitude              (delta_time) float64 dask.array<shape=(50045,), chunksize=(5000,)>
Data variables:
    atl06_quality_summary  (delta_time) int8 dask.array<shape=(50045,), chunksize=(5000,)>
    h_li                   (delta_time) float32 dask.array<shape=(50045,), chunksize=(5000,)>
    h_li_sigma             (delta_time) float32 dask.array<shape=(50045,), chunksize=(5000,)>
    segment_id             (delta_time) float64 dask.array<shape=(50045,), chunksize=(5000,)>
    sigma_geo_h            (delta_time) float32 dask.array<shape=(50045,), chunksize=(5000,)>
Attributes:
    Description:  The land_ice_height group contains the primary set of deriv...
    data_rate:    Data within this group

In [9]:
# convert to dataframe
df = ds.to_dataframe()

In [10]:
df.head()

,atl06_quality_summary,h_li,h_li_sigma,latitude,longitude,segment_id,sigma_geo_h
delta_time,,,,,,,
2019-02-15 18:01:21.878254460,1,650.075867,1.537900,59.503806,-129.207137,330757.0,100.087387
2019-02-15 18:01:21.881076856,1,651.325012,0.503594,59.503985,-129.207173,330758.0,100.067368
2019-02-15 18:01:21.920327520,1,651.900024,3.759399,59.506488,-129.207680,330772.0,100.159157
2019-02-15 18:01:21.923183560,1,653.157471,1.908529,59.506667,-129.207717,330773.0,100.025024
2019-02-15 18:01:21.931767076,1,651.198425,0.276231,59.507203,-129.207827,330776.0,100.002777


In [11]:
ds.to_dask_dataframe()

,delta_time,latitude,longitude,atl06_quality_summary,h_li,h_li_sigma,segment_id,sigma_geo_h
npartitions=11,,,,,,,,
0,datetime64[ns],float64,float64,int8,float32,float32,float64,float32
5000,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
50000,...,...,...,...,...,...,...,...
50044,...,...,...,...,...,...,...,...


In [12]:
# hvplot with all 50000 points
df.hvplot.scatter(x='delta_time', y='h_li', alpha=0.1, width=1000, height=500)

:Scatter   [delta_time]   (h_li)

In [13]:
# Map view with geoviews
crs = {'init': 'epsg:4326'}
gf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=crs)

gf.head()

,atl06_quality_summary,h_li,h_li_sigma,latitude,longitude,segment_id,sigma_geo_h,geometry
delta_time,,,,,,,,
2019-02-15 18:01:21.878254460,1,650.075867,1.537900,59.503806,-129.207137,330757.0,100.087387,POINT (-129.2071369154783 59.5038060410612)
2019-02-15 18:01:21.881076856,1,651.325012,0.503594,59.503985,-129.207173,330758.0,100.067368,POINT (-129.207173123747 59.50398484907924)
2019-02-15 18:01:21.920327520,1,651.900024,3.759399,59.506488,-129.207680,330772.0,100.159157,POINT (-129.2076801590167 59.50648820748679)
2019-02-15 18:01:21.923183560,1,653.157471,1.908529,59.506667,-129.207717,330773.0,100.025024,POINT (-129.2077165446327 59.50666700616529)
2019-02-15 18:01:21.931767076,1,651.198425,0.276231,59.507203,-129.207827,330776.0,100.002777,POINT (-129.2078271442943 59.50720333579383)


In [14]:
# Map View, plot every 100th's point
points = gf[::100].hvplot(geo=True, hover_cols=['h_li_sigma'], c='h_li', cmap='viridis', colorbar=True)
tiles = gv.tile_sources.StamenTerrain(width=800, height=500)

tiles * points

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (h_li,h_li_sigma)